### Sleep Disorder Data Ingestion


In [1]:
import os

In [2]:
%pwd

'd:\\Projects\\Sleeping_disorder_detection\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'd:\\Projects\\Sleeping_disorder_detection'

In [5]:
from dataclasses import dataclass
from pathlib import Path


@dataclass
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path

In [6]:
from sleep_disorder.constants import CONFIG_FILE_PATH, PARAMS_FILE_PATH
from sleep_disorder.utils.common import read_yaml, create_directories

In [7]:
class ConfigurationManager:
    def __init__(self, config_file_path=CONFIG_FILE_PATH, params_file_path=PARAMS_FILE_PATH):
        self.config = read_yaml(config_file_path)
        self.params = read_yaml(params_file_path)

        create_directories([self.config.artifacts_root])

    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
        )

        return data_ingestion_config

In [8]:
import urllib.request as request
from sleep_disorder.logging import logger
from sleep_disorder.utils.common import get_size

In [9]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config

        create_directories([self.config.root_dir])

    def get_data(self):
        try:
            if not os.path.exists(self.config.local_data_file):
                filename, headers = request.urlretrieve(self.config.source_URL, self.config.local_data_file)
                logger.info(f"File {filename} downloaded with following information:\n{headers}")
            else:
                logger.info(
                    f"File {self.config.local_data_file} already exists with size {get_size(Path(self.config.local_data_file))}"
                )
        except Exception as e:
            raise e

In [10]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.get_data()
except Exception as e:
    raise e

[2023-12-20 21:48:04,527]: INFO common yaml file: config\config.yaml loaded successfully.
[2023-12-20 21:48:04,532]: INFO common yaml file: params.yaml loaded successfully.
[2023-12-20 21:48:04,535]: INFO common Directory created at: artifacts
[2023-12-20 21:48:04,536]: INFO common Directory created at: artifacts/data_ingestion
[2023-12-20 21:48:05,868]: INFO 145822558 File artifacts/data_ingestion/local_data.csv downloaded with following information:
Connection: close
Content-Length: 23763
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: text/plain; charset=utf-8
ETag: "44401f1e0458228765c320259e36d8b1a273bcb3c200697e51c9c9d2858da41c"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: E9D2:30ABA0:E361C:1797D7:658313C3
Accept-Ranges: bytes
Date: Wed, 20 Dec 2023 16:18:13 GMT
Via: 1.1 varnish
X-Served-By: cache-del217